Detailed ad retrieval from 3 websites ingatlanok.hu,ingatlan.com, otthoncentrum.

Step 1: retrive unique ad's urls from the search engine result to 'urls' variable for a give district
Step 2: use the urls to retrive the ad homepage <body> tag
Step 3: save the ads into source specific .csv

In [1]:
    #urls
    url1a = 'https://ingatlan.com/szukites/elado+lakas+xi-ker+48-55-m2+ar-szerint?page='
    url1b = 'https://ingatlan.com/szukites/elado+lakas+vi-ker+38-55-m2+ar-szerint?page='
    url2 = 'https://ingatlanok.hu/elado/lakas+haz/budapest-v-ker:25nm-tol;33nm-ig?record=0&num=100'
    url2a = 'https://ingatlanok.hu/elado/lakas+haz/budapest-xi-ker:48nm-tol;55nm-ig?record=0&num=100'
    url2b = 'https://ingatlanok.hu/elado/lakas+haz/budapest-vi-ker:38nm-tol;55nm-ig?record=0&num=100'
    url3 = 'https://www.oc.hu/ingatlanok/lista/oldalszam:48/rendezes:relevance/felhasznalas:elado/jogi-status:hasznalt/tipus:flat/stilus:brick,/hely-ertek:budapest05-kerulet/hely-id:budapest05-kerulet,/netto-alapterulet:28~33/emelet:1,8'
    url3a = 'https://www.oc.hu/ingatlanok/lista/oldalszam:48/rendezes:relevance/felhasznalas:elado/jogi-status:hasznalt/tipus:flat/stilus:brick,/hely-ertek:budapest11-kerulet/hely-id:budapest11-kerulet,/netto-alapterulet:45~55/emelet:1,8'
    url3b = 'https://www.oc.hu/ingatlanok/lista/oldalszam:48/rendezes:relevance/felhasznalas:elado/jogi-status:hasznalt/tipus:flat/stilus:brick,/hely-ertek:budapest06-kerulet/hely-id:budapest06-kerulet,/netto-alapterulet:38~55/emelet:1,8'

    url1 = 'https://ingatlan.com/szukites/elado+lakas+v-ker+25-33-m2+ar-szerint?page='
    url2 = 'https://ingatlanok.hu/elado/lakas+haz/budapest-v-ker:25nm-tol;33nm-ig?record=0&num=100'
    url3 = 'https://www.oc.hu/ingatlanok/lista/oldalszam:48/rendezes:relevance/felhasznalas:elado/jogi-status:hasznalt/tipus:flat/stilus:brick,/hely-ertek:budapest05-kerulet/hely-id:budapest05-kerulet,/netto-alapterulet:28~33/emelet:1,8'
    


In [9]:
import requests
#from urllib.robotparser import RobotFileParser
from bs4 import BeautifulSoup
import urllib3
import warnings
warnings.filterwarnings('ignore')
import datetime as dtime
import time
import pandas as pd

In [10]:
#searchpage retrival

In [11]:
#ingatlan.com, multi-page
def _multi_get(url, n):
    '''
    gets multipages session based webpages like search results specialized to ingatlan.com site
    
    url for URL
    n for number of pages to follow
    
    returns ads specific url in list
    '''
    urls = []
    link = url + str(n)
    print(link)
    http = urllib3.PoolManager(headers={'User-Agent': 'Mozilla/5.0 (X11; Linux x86_64; rv:62.0) Gecko/20100101 Firefox/62.0'})
    response = http.request('GET', link)
    soup = BeautifulSoup(response.data, 'lxml')
    l = soup.find_all('div', attrs = {'class': 'listing js-listing listing--cluster-parent js-cluster-parent'})
    l = l + (soup.find_all('div', attrs = {'class': 'listing js-listing'}))
    for i in l:
            urls.append('https://ingatlan.com'+i.find('a')['href'])
    
    return urls

def multipagesSearch(url , v=10):
    '''
    wrapper for _multi_get(url, n) to introduce timer between page download 
    url for URL,
    v for number of pages to return
    
    _multi_get:
    gets multipages session based webpages like search results specialized to ingatlan.com site
    
    returns ad specific url in list
    '''
    i=1
    while i <= v:
        text = _multi_get(url, n=i)
        
        if 'csvtext' in locals():
            csvtext = csvtext + text
        else: csvtext = text
            
        time.sleep(1)
        i += 1
    return csvtext

#ingatlanok.hu, oc.hu, one page
def simpleSearch(url):
    '''
    gets simple one page website from the URL
    
    returns ad specific url for .hu and oc.hu pages
    '''
    r=requests.get(url, headers={'User-Agent': 'Mozilla/5.0 (X11; Linux x86_64; rv:62.0) Gecko/20100101 Firefox/62.0'})
    
    page = BeautifulSoup(r.content, 'lxml')
    body = page.body
    url_list = []
    if 'ingatlanok.hu' in url:
        a = body.find_all('div', attrs={'class': 'result-row'})
        for i in a:
            url_list.append(i.find_all('a')[1]['href'])
    if 'oc.hu' in url:
        a = body.find_all('div', attrs={'class': 'estate-list-box'})
        for i in a:
            url_list.append('https://www.oc.hu'+i.find_all('a')[0]['href'])
    return url_list

In [12]:
#compose for searPage url retrieval
def searchPage(district):
    '''
    retrives the search page results ad's url into a [list] according to the given district.
    Options:
    District: 'V', 'VI','XI'
    '''
    if district == 'V':
        url1 = 'https://ingatlan.com/szukites/elado+lakas+v-ker+25-33-m2+ar-szerint?page='
        url2 = 'https://ingatlanok.hu/elado/lakas+haz/budapest-v-ker:25nm-tol;33nm-ig?record=0&num=100'
        url3 = 'https://www.oc.hu/ingatlanok/lista/oldalszam:48/rendezes:relevance/felhasznalas:elado/jogi-status:hasznalt/tipus:flat/stilus:brick,/hely-ertek:budapest05-kerulet/hely-id:budapest05-kerulet,/netto-alapterulet:28~33/emelet:1,8'
    elif district == 'VI':
        url1 = 'https://ingatlan.com/szukites/elado+lakas+vi-ker+38-55-m2+ar-szerint?page='
        url2 = 'https://ingatlanok.hu/elado/lakas+haz/budapest-vi-ker:38nm-tol;55nm-ig?record=0&num=100'
        url3 = 'https://www.oc.hu/ingatlanok/lista/oldalszam:48/rendezes:relevance/felhasznalas:elado/jogi-status:hasznalt/tipus:flat/stilus:brick,/hely-ertek:budapest06-kerulet/hely-id:budapest06-kerulet,/netto-alapterulet:38~55/emelet:1,8'
    elif district == 'XI':
        url1 = 'https://ingatlan.com/szukites/elado+lakas+xi-ker+48-55-m2+ar-szerint?page='
        url2 = 'https://ingatlanok.hu/elado/lakas+haz/budapest-xi-ker:48nm-tol;55nm-ig?record=0&num=100'
        url3 = 'https://www.oc.hu/ingatlanok/lista/oldalszam:48/rendezes:relevance/felhasznalas:elado/jogi-status:hasznalt/tipus:flat/stilus:brick,/hely-ertek:budapest11-kerulet/hely-id:budapest11-kerulet,/netto-alapterulet:45~55/emelet:1,8'
    
    hu = simpleSearch(url2)
    print('Hu search: ',len(hu))
    
    com = multipagesSearch(url1)
    print('Com search: ',len(com))
    
    cen = simpleSearch(url3)
    print('Cen search: ',len(cen))
    
    alls = [*hu,*com,*cen]
    print('All urls: ', len(alls))
    
    alls =[i for i in alls if 'http' in i]
    return alls

In [13]:
# detailed webpage retrival

In [14]:
def _simple(url):
    '''
    Returns the page <body> tag from url retrieval
    '''
    page = requests.get(url, headers={'User-Agent': 'Mozilla/5.0 (X11; Linux x86_64; rv:62.0) Gecko/20100101 Firefox/62.0'})
    body = BeautifulSoup(page.content, 'lxml').body

    return body

def details(urls, district):
    '''
    Go through the list of urls, retrieves the body tag. 
    Sorted by url, saves the body tag of those that has the keyword specific to the url source 
    
    district: 'V', 'VI', 'XI'
    '''
    
    adshu = []
    adscom = []
    adscen = []
    n=0
    for i in urls:
        print(n,'\t',i)
        if 'ingatlanok.hu' in i:
            try:
                back = simple(i)               
                txt = back.get_text()

                if 'Azonosító:' in txt:             
                    adshu.append(back)
                    
            except: pass
            
        elif 'oc.hu' in i:
            try:
                back = simple(i)               
                txt = back.get_text()
                
                if 'Regisztrációs szám:' in txt:             
                    adscen.append(back)
            except: pass
            
        else:
            try:
                back = simple(i)               
                txt = back.get_text()
                
                if 'Parkolás' in txt or 'Ingatlan állapota' in txt:             
                    adscom.append(back)
            except: pass
        
        n+=1 # counter
    
    now = dtime.datetime.now()
    m = [str(now.month) if len(str(now.month)) == 2 else '0' + str(now.month)]
    d = [str(now.day) if len(str(now.day)) == 2 else '0' + str(now.day)]
    y = str(now.year)
    
    with open('adshu'+ district + '_' + d[0] + m[0] + y + '.csv', 'w') as f:
                f.write('%s' % adshu)
    with open('adscom'+ district + '_' + d[0] + m[0] + y + '.csv', 'w') as f:
                f.write('%s' % adscom)
    with open('adscen'+ district + '_' + d[0] + m[0] + y + '.csv', 'w') as f:
                f.write('%s' % adscen)
    return

In [15]:
def checkDownload(date, d = ['V','VI','XI']):
    '''
    Check the No of saved ads.
    Search by date
    Search by district in list format -> ['XX']
    '''
    for n in d:
        for i in ['adshu'+n,'adscom'+n,'adscen'+n]:
            try:
                f = i+ '_' + date +'.csv'
                with open(f, 'r') as r:
                    l = r.read()
                ll = l.split('</body>,')
                print('Saved ads in {}: {}'.format(i,len(ll)))
            except: print('No such file ' + f)

In [16]:
for i in ['V','VI', 'XI']:
    urls = searchPage(i)
    details(urls,i)

Hu search:  19
https://ingatlan.com/szukites/elado+lakas+v-ker+25-33-m2+ar-szerint?page=1
https://ingatlan.com/szukites/elado+lakas+v-ker+25-33-m2+ar-szerint?page=2
https://ingatlan.com/szukites/elado+lakas+v-ker+25-33-m2+ar-szerint?page=3
https://ingatlan.com/szukites/elado+lakas+v-ker+25-33-m2+ar-szerint?page=4
https://ingatlan.com/szukites/elado+lakas+v-ker+25-33-m2+ar-szerint?page=5
https://ingatlan.com/szukites/elado+lakas+v-ker+25-33-m2+ar-szerint?page=6
https://ingatlan.com/szukites/elado+lakas+v-ker+25-33-m2+ar-szerint?page=7
https://ingatlan.com/szukites/elado+lakas+v-ker+25-33-m2+ar-szerint?page=8
https://ingatlan.com/szukites/elado+lakas+v-ker+25-33-m2+ar-szerint?page=9
https://ingatlan.com/szukites/elado+lakas+v-ker+25-33-m2+ar-szerint?page=10
Com search:  52
Cen search:  3
All urls:  74
0 	 https://ingatlanok.hu/elado/lakas/budapest_v_ker/36-millio-ft;32-negyzetmeter;1-szoba;tegla-epitesu/9737619
1 	 https://ingatlanok.hu/elado/lakas/budapest_v_ker/36-millio-ft;32-negyzetm

70 	 https://ingatlan.com/v-ker/elado+lakas/tegla-epitesu-lakas/budapest+5+kerulet+marko+utca/31029475
71 	 https://www.oc.hu/ingatlanok/LK503373?c=1b06e8e0
72 	 https://www.oc.hu/ingatlanok/H397307?c=1b06e8e0
73 	 https://www.oc.hu/ingatlanok/H391787?c=1b06e8e0
Hu search:  100
https://ingatlan.com/szukites/elado+lakas+vi-ker+38-55-m2+ar-szerint?page=1
https://ingatlan.com/szukites/elado+lakas+vi-ker+38-55-m2+ar-szerint?page=2
https://ingatlan.com/szukites/elado+lakas+vi-ker+38-55-m2+ar-szerint?page=3
https://ingatlan.com/szukites/elado+lakas+vi-ker+38-55-m2+ar-szerint?page=4
https://ingatlan.com/szukites/elado+lakas+vi-ker+38-55-m2+ar-szerint?page=5
https://ingatlan.com/szukites/elado+lakas+vi-ker+38-55-m2+ar-szerint?page=6
https://ingatlan.com/szukites/elado+lakas+vi-ker+38-55-m2+ar-szerint?page=7
https://ingatlan.com/szukites/elado+lakas+vi-ker+38-55-m2+ar-szerint?page=8
https://ingatlan.com/szukites/elado+lakas+vi-ker+38-55-m2+ar-szerint?page=9
https://ingatlan.com/szukites/elado+l

62 	 https://ingatlanok.hu/elado/lakas/budapest_vi_ker/60-millio-ft;47-negyzetmeter;2-szoba;tegla-epitesu/9956353
63 	 https://ingatlanok.hu/elado/lakas/budapest_vi_ker/42-millio-ft;44-negyzetmeter;1-1-szoba;tegla-epitesu/9918823
64 	 https://ingatlanok.hu/elado/lakas/budapest_vi_ker/65-millio-ft;52-negyzetmeter;2-szoba;tegla-epitesu/9915839
65 	 https://ingatlanok.hu/elado/lakas/budapest_vi_ker/34-millio-ft;43-negyzetmeter;1-1-szoba;tegla-epitesu/9925809
66 	 https://ingatlanok.hu/elado/lakas/budapest_vi_ker/39-millio-ft;39-negyzetmeter;1-szoba;tegla-epitesu/9950936
67 	 https://ingatlanok.hu/elado/lakas/budapest_vi_ker/32-millio-ft;38-negyzetmeter;1-szoba;tegla-epitesu/9950992
68 	 https://ingatlanok.hu/elado/lakas/budapest_vi_ker/49-millio-ft;47-negyzetmeter;2-szoba;tegla-epitesu/9725942
69 	 https://ingatlanok.hu/elado/lakas/budapest_vi_ker/47-millio-ft;44-negyzetmeter;1-szoba;tegla-epitesu/9909898
70 	 https://ingatlanok.hu/elado/lakas/budapest_vi_ker/37-millio-ft;44-negyzetmeter;

138 	 https://ingatlan.com/vi-ker/elado+lakas/tegla-epitesu-lakas/budapest+6+kerulet+rozsa+utca/30712066
139 	 https://ingatlan.com/vi-ker/elado+lakas/tegla-epitesu-lakas/budapest+6+kerulet+szondi+utca/30297026
140 	 https://ingatlan.com/vi-ker/elado+lakas/tegla-epitesu-lakas/budapest+6+kerulet+sziv+utca/29174315
141 	 https://ingatlan.com/vi-ker/elado+lakas/tegla-epitesu-lakas/budapest+6+kerulet+sziv+utca/31380085
142 	 https://ingatlan.com/vi-ker/elado+lakas/tegla-epitesu-lakas/budapest+6+kerulet+sziv+utca/30846742
143 	 https://ingatlan.com/vi-ker/elado+lakas/tegla-epitesu-lakas/budapest+6+kerulet/30689668
144 	 https://ingatlan.com/vi-ker/elado+lakas/tegla-epitesu-lakas/budapest+6+kerulet+sziv+utca+3-5/29971898
145 	 https://ingatlan.com/vi-ker/elado+lakas/tegla-epitesu-lakas/budapest+6+kerulet+lovolde+ter/29663924
146 	 https://ingatlan.com/vi-ker/elado+lakas/tegla-epitesu-lakas/budapest+6+kerulet+terez+korut/30549742
147 	 https://ingatlan.com/vi-ker/elado+lakas/tegla-epitesu-lak

217 	 https://ingatlan.com/vi-ker/elado+lakas/tegla-epitesu-lakas/budapest+6+kerulet+nagymezo+utca/31138294
218 	 https://ingatlan.com/vi-ker/elado+lakas/tegla-epitesu-lakas/budapest+6+kerulet+szinyei+merse+pal+utca/30145886
219 	 https://ingatlan.com/vi-ker/elado+lakas/tegla-epitesu-lakas/budapest+6+kerulet/30245693
220 	 https://ingatlan.com/vi-ker/elado+lakas/tegla-epitesu-lakas/budapest+6+kerulet+rippl-ronai+utca/31429882
221 	 https://ingatlan.com/vi-ker/elado+lakas/tegla-epitesu-lakas/budapest+6+kerulet+vorosmarty+utca/27571074
222 	 https://ingatlan.com/vi-ker/elado+lakas/tegla-epitesu-lakas/budapest+6+kerulet+eotvos+utca/31230232
223 	 https://ingatlan.com/vi-ker/elado+lakas/tegla-epitesu-lakas/budapest+6+kerulet+eotvos+utca/31372195
224 	 https://ingatlan.com/vi-ker/elado+lakas/tegla-epitesu-lakas/budapest+6+kerulet+eotvos+utca/31350850
225 	 https://ingatlan.com/vi-ker/elado+lakas/tegla-epitesu-lakas/budapest+6+kerulet+rippl-ronai+utca/31421328
226 	 https://ingatlan.com/vi-k

298 	 https://ingatlan.com/vi-ker/elado+lakas/tegla-epitesu-lakas/budapest+6+kerulet+szondi+utca/31373284
299 	 https://ingatlan.com/vi-ker/elado+lakas/tegla-epitesu-lakas/budapest+6+kerulet+szondi+utca/31206532
300 	 https://www.oc.hu/ingatlanok/LK502176?c=79cc83b7
301 	 https://www.oc.hu/ingatlanok/LK479686?c=79cc83b7
302 	 https://www.oc.hu/ingatlanok/H405942?c=79cc83b7
303 	 https://www.oc.hu/ingatlanok/H404476?c=79cc83b7
304 	 https://www.oc.hu/ingatlanok/H401438?c=79cc83b7
305 	 https://www.oc.hu/ingatlanok/H397999?c=79cc83b7
306 	 https://www.oc.hu/ingatlanok/H397869?c=79cc83b7
307 	 https://www.oc.hu/ingatlanok/H394363?c=79cc83b7
308 	 https://www.oc.hu/ingatlanok/H404024?c=79cc83b7
309 	 https://www.oc.hu/ingatlanok/H401440?c=79cc83b7
310 	 https://www.oc.hu/ingatlanok/H405053?c=79cc83b7
311 	 https://www.oc.hu/ingatlanok/H403733?c=79cc83b7
312 	 https://www.oc.hu/ingatlanok/H403737?c=79cc83b7
313 	 https://www.oc.hu/ingatlanok/H403744?c=79cc83b7
314 	 https://www.oc.hu/ingatl

39 	 https://ingatlanok.hu/elado/lakas/budapest_xi_ker/44-millio-ft;55-negyzetmeter;2-szoba;tegla-epitesu/9901079
40 	 https://ingatlanok.hu/elado/lakas/budapest_xi_ker/34-millio-ft;50-negyzetmeter;2-szoba;panel-epitesu/9681631
41 	 https://ingatlanok.hu/elado/lakas/budapest_xi_ker/32-millio-ft;48-negyzetmeter;2-szoba;panel-epitesu/9875100
42 	 https://ingatlanok.hu/elado/lakas/budapest_xi_ker/34-millio-ft;50-negyzetmeter;2-szoba;panel-epitesu/9681736
43 	 https://ingatlanok.hu/elado/lakas/budapest_xi_ker/49-millio-ft;50-negyzetmeter;2-szoba;tegla-epitesu/9934637
44 	 https://ingatlanok.hu/elado/lakas/budapest_xi_ker/32-millio-ft;53-negyzetmeter;1-2-szoba;panel-epitesu/9891515
45 	 https://ingatlanok.hu/elado/lakas/budapest_xi_ker/70-millio-ft;55-negyzetmeter;1-2-szoba;tegla-epitesu/9943530
46 	 https://ingatlanok.hu/elado/lakas/budapest_xi_ker/61-millio-ft;52-negyzetmeter;1-1-szoba;tegla-epitesu/9900949
47 	 https://ingatlanok.hu/elado/lakas/budapest_xi_ker/29-millio-ft;48-negyzetmete

113 	 https://ingatlan.com/xi-ker/elado+lakas/panel-lakas/budapest+11+kerulet+igmandi+utca+5/29618399
114 	 https://ingatlan.com/xi-ker/elado+lakas/tegla-epitesu-lakas/budapest+11+kerulet+korosy+jozsef+utca/31304200
115 	 https://ingatlan.com/xi-ker/elado+lakas/panel-lakas/budapest+11+kerulet+safrany+utca/31235212
116 	 https://ingatlan.com/xi-ker/elado+lakas/panel-lakas/budapest+11+kerulet+etele+ut/31436298
117 	 https://ingatlan.com/xi-ker/elado+lakas/panel-lakas/budapest+11+kerulet+andor+utca/31324801
118 	 https://ingatlan.com/xi-ker/elado+lakas/panel-lakas/budapest+11+kerulet+etele+ut/31387834
119 	 https://ingatlan.com/xi-ker/elado+lakas/panel-lakas/budapest+11+kerulet+etele+ut/31325443
120 	 https://ingatlan.com/xi-ker/elado+lakas/panel-lakas/budapest+11+kerulet+andor+utca/31213351
121 	 https://ingatlan.com/xi-ker/elado+lakas/panel-lakas/budapest+11+kerulet+menyecske+utca/31084642
122 	 https://ingatlan.com/xi-ker/elado+lakas/panel-lakas/budapest+11+kerulet+andor+utca+11a/30987

199 	 https://ingatlan.com/xi-ker/elado+lakas/tegla-epitesu-lakas/budapest+11+kerulet+budaorsi+ut/29681699
200 	 https://ingatlan.com/xi-ker/elado+lakas/panel-lakas/budapest+11+kerulet+allende+park/31422699
201 	 https://ingatlan.com/xi-ker/elado+lakas/panel-lakas/budapest+11+kerulet/31436025
202 	 https://ingatlan.com/xi-ker/elado+lakas/panel-lakas/budapest+11+kerulet/31422460
203 	 https://ingatlan.com/xi-ker/elado+lakas/panel-lakas/budapest+11+kerulet/31383394
204 	 https://ingatlan.com/xi-ker/elado+lakas/panel-lakas/budapest+11+kerulet+safrany+utca/31375084
205 	 https://ingatlan.com/xi-ker/elado+lakas/panel-lakas/budapest+11+kerulet/31157335
206 	 https://ingatlan.com/xi-ker/elado+lakas/panel-lakas/budapest+11+kerulet+safrany+utca/31047616
207 	 https://ingatlan.com/xi-ker/elado+lakas/panel-lakas/budapest+11+kerulet/31342264
208 	 https://ingatlan.com/xi-ker/elado+lakas/panel-lakas/budapest+11+kerulet+fegyvernek+utca/31368397
209 	 https://ingatlan.com/xi-ker/elado+lakas/panel-lak

283 	 https://ingatlan.com/xi-ker/elado+lakas/tegla-epitesu-lakas/budapest+11+kerulet+bartok+bela+ut/31333441
284 	 https://ingatlan.com/xi-ker/elado+lakas/panel-lakas/budapest+11+kerulet/31275079
285 	 https://ingatlan.com/xi-ker/elado+lakas/tegla-epitesu-lakas/budapest+11+kerulet+bartok+bela+ut/31232041
286 	 https://ingatlan.com/xi-ker/elado+lakas/tegla-epitesu-lakas/budapest+11+kerulet+bartok+bela+ut/31220035
287 	 https://ingatlan.com/xi-ker/elado+lakas/tegla-epitesu-lakas/budapest+11+kerulet+bartok+bela+ut/31215928
288 	 https://ingatlan.com/xi-ker/elado+lakas/tegla-epitesu-lakas/budapest+11+kerulet+bartok+bela+ut/31210207
289 	 https://ingatlan.com/xi-ker/elado+lakas/panel-lakas/budapest+11+kerulet/31174108
290 	 https://ingatlan.com/xi-ker/elado+lakas/panel-lakas/budapest+11+kerulet/31159744
291 	 https://ingatlan.com/xi-ker/elado+lakas/panel-lakas/budapest+11+kerulet/31154713
292 	 https://ingatlan.com/xi-ker/elado+lakas/tegla-epitesu-lakas/budapest+11+kerulet+bartok+bela+ut+5

In [17]:
import os
for i in sorted(os.listdir()):
    if 'ads' in i:
        print(i,round(os.stat(i).st_size/(1024*1024),2), 'M')

adscenVI_03032020.csv 5.18 M
adscenVI_06042020.csv 5.25 M
adscenVI_08032020.csv 5.19 M
adscenVI_15032020.csv 5.23 M
adscenVI_22032020.csv 5.24 M
adscenVI_30032020.csv 5.25 M
adscenV_01032020.csv 0.2 M
adscenV_06042020.csv 0.3 M
adscenV_08032020.csv 0.2 M
adscenV_15032020.csv 0.2 M
adscenV_24022020.csv 0.18 M
adscenV_30032020.csv 0.3 M
adscenXI_04032020.csv 2.96 M
adscenXI_06042020.csv 2.78 M
adscenXI_08032020.csv 3.06 M
adscenXI_15032020.csv 2.99 M
adscenXI_22032020.csv 2.9 M
adscenXI_30032020.csv 2.78 M
adscomVI_03032020.csv 17.44 M
adscomVI_06042020.csv 17.77 M
adscomVI_08032020.csv 17.5 M
adscomVI_15032020.csv 17.52 M
adscomVI_22032020.csv 17.41 M
adscomVI_30032020.csv 17.46 M
adscomV_01032020.csv 4.04 M
adscomV_06042020.csv 4.65 M
adscomV_08032020.csv 3.86 M
adscomV_15032020.csv 4.14 M
adscomV_22032020.csv 4.38 M
adscomV_24022020.csv 4.11 M
adscomV_30032020.csv 4.4 M
adscomXI_04032020.csv 17.51 M
adscomXI_06042020.csv 17.73 M
adscomXI_08032020.csv 17.49 M
adscomXI_15032020.csv 17.5

In [14]:
checkDownload('30032020', ['XI'])

Saved ads in adshuXI: 100
Saved ads in adscomXI: 200
Saved ads in adscenXI: 25
